In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import os
import datetime
import itertools

# ログ出力用の関数
def printTime(tag):
    print(tag + ':', datetime.datetime.now())

printTime('Inputファイルを表示')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Inputファイルを表示: 2020-02-01 21:09:42.594745
/kaggle/input/test.csv
/kaggle/input/data_description.txt
/kaggle/input/train.csv
/kaggle/input/.ipynb_checkpoints/train-checkpoint.csv
/kaggle/input/.ipynb_checkpoints/test-checkpoint.csv
/kaggle/input/.ipynb_checkpoints/data_description-checkpoint.txt


#### 学習データの読み込み

In [120]:
train = pd.read_csv('/kaggle/input/train.csv')
test = pd.read_csv('/kaggle/input/test.csv')

# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['SalePrice'], axis=1)
train_y = train['SalePrice']

# テストデータは特徴量のみなので、そのままでよい
test_x = test.copy()

 #### 特徴量作成

In [121]:
# 解析対象とする数値列を設定
targets_num = ['1stFlrSF', 
               '2ndFlrSF'
               'BsmtFinSF1', 
               'Fireplaces', 
               'FullBath', 
               'GarageArea', 
               'GarageCars', 
               'GarageYrBlt', 
               'GrLivArea', 
               'MasVnrArea', 
               'OverallQual', 
               'OverallCond',
               'TotalBsmtSF', 
               'TotRmsAbvGrd', 
               'YearBuilt', 
               'YearRemodAdd']

# 解析対象とするカテゴリ列を設定

targets_category = [
                    'MSSubClass',
                    'MSZoning',
                    #'Street',
                    #'Alley',
                    'LotShape',
                    'LandContour',
                    #'Utilities',
                    'LotConfig'
                    #'LandSlope',
                    'Neighborhood',
                    'Condition1',
                    #'Condition2',
                    'BldgType',
                    'HouseStyle',
                    'RoofStyle',
                    'RoofMatl',
                    'Exterior1st',
                    'Exterior2nd',
                    'MasVnrType',
                    'ExterQual',
                    'ExterCond',
                    'Foundation',
                    'BsmtQual',
                    'BsmtCond',
                    'BsmtExposure',
                    'BsmtFinType1',
                    'BsmtFinType2',
                    #'Heating',
                    'HeatingQC',
                    #'CentralAir',
                    'Electrical',
                    'KitchenQual',
                    'Functional',
                    'FireplaceQu',
                    'GarageType',
                    'GarageFinish',
                    'GarageQual',
                    'GarageCond',
                    #'PavedDrive',
                    'PoolQC',
                    'Fence',
                    'MiscFeature',
                    'SaleType',
                    'SaleCondition']

# 解析対象とする数値列とカテゴリ列の列名を連結
targets = targets_num
targets.extend(targets_category)

# 解析対象列のみを抽出
train_x = train_x.reindex(columns=targets)
test_x = test_x.reindex(columns=targets)

# Dataframeの列の型がobjectのままだとLabelEncoderでエラーするため、strに変換
# ※変換処理後にinfo()で型を確認しても、objectと表示されるが内部的にはstrに変換されている。
train_x[targets_category] = train_x[targets_category].astype(str)
test_x[targets_category] = test_x[targets_category].astype(str)

# ラベルエンコーディング用に学習データとテストデータをunion
lavel_x = train_x.append(test_x, ignore_index=True)

# カテゴリ変数をlabel encodingする
for c in targets_category:
    # 学習データとテストデータに基づいてラベルを設定
    le = LabelEncoder()
    le.fit(lavel_x[c])
    train_x[c] = le.transform(train_x[c])
    test_x[c] = le.transform(test_x[c])


#### モデル作成

In [131]:
printTime('モデルの作成開始')

# ハイパーパラメータの設定
prm_n_estimators=20
prm_random_state=71
prm_max_depth=5
prm_min_child_weight=0.4

# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=prm_n_estimators, 
                      random_state=prm_random_state, 
                      max_depth=prm_max_depth, 
                      min_child_weight=prm_min_child_weight)

model.fit(train_x, train_y)

# テストデータの予測値を出力する
pred = model.predict(test_x)

# 提出用ファイルの作成
submission = pd.DataFrame({'Id': test['Id'], 'SalePrice': pred})
submission.to_csv('/kaggle/output/submission_first.csv', index=False)

printTime('モデルの作成終了')

モデルの作成開始: 2020-02-01 22:23:00.849068
モデルの作成終了: 2020-02-01 22:23:39.830302


#### バリデーション

In [132]:
printTime('バリデーション開始')

# 各foldのスコアを保存するリスト
scores_rmse = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=prm_n_estimators, 
                          random_state=prm_random_state, 
                          max_depth=prm_max_depth, 
                          min_child_weight=prm_min_child_weight)
    
    model.fit(tr_x, tr_y)

    # バリデーションデータの予測値を出力する
    va_pred = model.predict(va_x)

    # バリデーションデータでのスコア(真の値の対数と予測値の対数の二乗平均平方根誤差 (RMSE))を計算する
    rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))

    # そのfoldのスコアを保存する
    scores_rmse.append(rmse)

# 各foldのスコアの平均を出力する
rmse = np.mean(scores_rmse)
print(f'RMSE: {rmse:.5f}')

printTime('バリデーション終了')

バリデーション開始: 2020-02-01 22:23:45.717642
RMSE: 0.23061
バリデーション終了: 2020-02-01 22:24:57.195893


#### モデルチューニング

In [130]:
printTime('モデルチューニング開始')

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [0.3, 0.4, 0.5]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# ループカウント変数
loopcount = 0

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

    loopcount += 1
    
    score_folds = []
    # クロスバリデーションを行う
    # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict(va_x)
        rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))
        score_folds.append(rmse)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

    printTime(f'max_depth: {max_depth}, '\
              f'min_child_weight: {min_child_weight}, '\
              f'score: {score_mean:.5f}')
    
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

printTime('モデルチューニング終了')

モデルチューニング開始: 2020-02-01 22:12:07.972415
max_depth: 4, min_child_weight: 0.3, score: 0.24143: 2020-02-01 22:13:19.784720
max_depth: 4, min_child_weight: 0.4, score: 0.23398: 2020-02-01 22:14:29.766758
max_depth: 4, min_child_weight: 0.5, score: 0.23479: 2020-02-01 22:15:38.912292
max_depth: 5, min_child_weight: 0.3, score: 0.23888: 2020-02-01 22:16:50.670331
max_depth: 5, min_child_weight: 0.4, score: 0.22959: 2020-02-01 22:18:01.805262
max_depth: 5, min_child_weight: 0.5, score: 0.23286: 2020-02-01 22:19:11.508185
max_depth: 6, min_child_weight: 0.3, score: 0.24351: 2020-02-01 22:20:23.518411
max_depth: 6, min_child_weight: 0.4, score: 0.23014: 2020-02-01 22:21:34.432612
max_depth: 6, min_child_weight: 0.5, score: 0.23176: 2020-02-01 22:22:44.381376
max_depth: 5, min_child_weight: 0.4
モデルチューニング終了: 2020-02-01 22:22:44.381968
